In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/captchadata/temp'):
    for filename in filenames:
        a = 1
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os

lst = os.listdir('../input/temp')

In [4]:
len(lst)

10000

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization,LSTM, Reshape, SimpleRNN
from keras.layers.recurrent import GRU

Using TensorFlow backend.


In [7]:
import os
import string
import cv2

In [8]:
characters = string.ascii_lowercase+"0123456789"+string.ascii_uppercase
characters

'abcdefghijklmnopqrstuvwxyz0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [9]:
len(characters)

62

In [10]:
all_images =  os.listdir('../input/temp')

In [11]:
all_images[-1], len(all_images)

('01243u_36d486a8-5908-4a67-930b-828f7a2c39f7.png', 10000)

In [12]:
def preprocess_data():
    
    inp_x = np.zeros((len(all_images), 100,160,1))
    inp_y = np.zeros((len(all_images), 6,len(characters)))
    for idx, image in enumerate(all_images):
        # ----------------prepare input container
        
        
        img = cv2.imread("../input/temp/"+image, cv2.IMREAD_GRAYSCALE)
        #print(np.max(img))
        #plt.imshow(img)
        #print(img.shape)
        
        # ---------------------Scale images --------------
        
        img = img/255.0
        
        image_txt = image[0:6]
        target_oht = np.zeros((5,len(characters)))
        if(len(image_txt)<7):
            img = np.reshape(img, (100,160,1))
            inp_x[idx] = img
            
             # ------------------Define targets and code them using OneHotEncoding
            target_oht = np.zeros((6,len(characters)))
            for k, char in enumerate(image_txt):
                target_oht[k, characters.find(char)] = 1
            inp_y[idx] = target_oht
    return inp_x, inp_y
        
               
        
        
        

In [13]:
X, Y = preprocess_data()

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_x ,test_x, train_y ,test_y = train_test_split(X,Y, shuffle = True, test_size = .1)

In [16]:
train_x.shape,train_y.shape, test_x.shape,test_y.shape

((9000, 100, 160, 1), (9000, 6, 62), (1000, 100, 160, 1), (1000, 6, 62))

In [27]:
model = Sequential()

In [28]:
def prepare_model():
    model.add(Conv2D(128, (3,3), input_shape = (100,160,1) ,padding = 'same', activation = 'relu'))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same',))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    #model.add(MaxPooling2D(pool_size =(3,3),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (3,3),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'relu', padding = 'same'))
  
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (3,3),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (1,1),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (1,1),padding ='same'))
    print(model.output_shape)
    model.add(Reshape((model.output_shape[1], model.output_shape[2]*model.output_shape[3])))
    print(model.output_shape)
    #model.add(LSTM(50, batch_input_shape=(2,5,36),stateful = True, return_sequences = True))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
  


    model.add(Dense(62, activation = "softmax"))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
   
    return model
    

In [29]:
model =prepare_model()

(None, 6, 9, 32)
(None, 6, 288)


In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 100, 160, 128)     1280      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 50, 80, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 50, 80, 64)        73792     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 80, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 27, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 17, 27, 32)        18464     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 9, 32)         

In [31]:
#model = create_model()
hist = model.fit(train_x, train_y, batch_size=32, epochs=50,verbose=1, validation_data=(test_x, test_y))

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 11s 1ms/step - loss: 1.7058 - accuracy: 0.5376 - val_loss: 1.5678 - val_accuracy: 0.5650
Epoch 2/50
9000/9000 [==============================] - 10s 1ms/step - loss: 1.5651 - accuracy: 0.5596 - val_loss: 1.5597 - val_accuracy: 0.5632
Epoch 3/50
9000/9000 [==============================] - 10s 1ms/step - loss: 1.5584 - accuracy: 0.5603 - val_loss: 1.5575 - val_accuracy: 0.5637
Epoch 4/50
9000/9000 [==============================] - 10s 1ms/step - loss: 1.5564 - accuracy: 0.5597 - val_loss: 1.5588 - val_accuracy: 0.5590
Epoch 5/50
9000/9000 [==============================] - 10s 1ms/step - loss: 1.5542 - accuracy: 0.5583 - val_loss: 1.5552 - val_accuracy: 0.5592
Epoch 6/50
9000/9000 [==============================] - 10s 1ms/step - loss: 1.5527 - accuracy: 0.5605 - val_loss: 1.5541 - val_accuracy: 0.5580
Epoch 7/50
9000/9000 [==============================] - 10s 1ms/step - loss: 1.551

In [32]:
score= model.evaluate(test_x,test_y,verbose=1)
print('Test Loss and accuracy:', score)

1000/1000 [==============================] - 0s 374us/step
Test Loss and accuracy: [0.531450226187706, 0.8568333387374878]


In [33]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print("img",img.shape)
    if(img is not None):
        img = cv2.resize(img, (160,100))
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (6, 62))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += characters[l]
    return capt#, sum(probs) / 5

In [34]:
for i in all_images[:20]:

    #model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
    print(i[:6],"predicted : ",predict('../input/temp/'+i))
    

0125yn predicted :  0125yn
0125Bg predicted :  0125Bg
01247N predicted :  01247N
0125HG predicted :  0125AG
0125fk predicted :  0125Hk
0124yB predicted :  0124V8
0124IC predicted :  0124Jc
0125p8 predicted :  0125q8
0123yX predicted :  0123yX
0125lz predicted :  0125lZ
0123PO predicted :  0123PO
0123ST predicted :  0123sT
01246c predicted :  01246c
0124Tm predicted :  0124Tm
0124JD predicted :  0124JD
0125pa predicted :  0125pa
0123Jp predicted :  0123Up
0124ix predicted :  0124JX
0124nu predicted :  0124nl
01249i predicted :  01249i


In [36]:
# --------------------Saving Model --------------------------------------


model.save_weights('../kaggle/working/model_weights.h5')

# Save the model architecture
with open('../kaggle/working/model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [37]:
# ------------------------------Loading ---------------------------------

from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'model_architecture.json'